In [8]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [9]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Sat, 17 Aug 2019 10:23:02 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '15033', 'Connection': 'keep-alive', 'Server': 'mw1262.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Content-language': 'en', 'Last-Modified': 'Fri, 02 Aug 2019 12:59:51 GMT', 'Backend-Timing': 'D=94406 t=1565960391240238', 'Vary': 'Accept-Encoding,Cookie,Authorization,X-Seven', 'Content-Encoding': 'gzip', 'X-Varnish': '676595261 10761119, 521550661 227090208', 'Age': '76991', 'X-Cache': 'cp1081 hit/8, cp1089 hit/18', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=17-Aug-2019;Path=/;HttpOnly;secure;Expires=Wed, 18 Sep 2019 00:

## scrape wikipedia

In [10]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

## Locate Table and use tags to find postal code by Borough and Neighbourhood

In [11]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)

(211, 3)


In [12]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [13]:
df.loc[df['PostalCode'] == 'M5A']


,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [15]:
df.shape

(103, 3)

In [18]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [19]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [20]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [21]:
df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [22]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [26]:
CLIENT_ID = 'HRLI4MFHLW3AAVJWYEIV0S5FMYQGXOQ4X5KUR4ITSLQMHDTA' # Foursquare ID
CLIENT_SECRET = 'XEQ4N04HUCELGZXLDDIVHN5VVNRCUYZR3JJ5SIFQ30JKPFAB' # Foursquare Secret
VERSION = '20180605' # API version

In [29]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

KeyError: "None of ['PostalCode'] are in the columns"

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HRLI4MFHLW3AAVJWYEIV0S5FMYQGXOQ4X5KUR4ITSLQMHDTA&client_secret=XEQ4N04HUCELGZXLDDIVHN5VVNRCUYZR3JJ5SIFQ30JKPFAB&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [31]:
results = requests.get(url).json()

In [32]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4b04a05bf964a520c45522e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b04a05bf964a520c45522e3,Sheriff's No Frills,1530 Albion Rd,at Finch Ave. W.,43.741968,-79.586639,"[{'label': 'display', 'lat': 43.74196806166214...",318,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
1,e-0-4be58dc4cf200f479154133c-1,0,"[{'summary': 'This spot is popular', 'type': '...",4be58dc4cf200f479154133c,Shoppers Drug Mart,1530 Albion Rd,Albion Mall,43.740832,-79.583347,"[{'label': 'display', 'lat': 43.74083239297218...",438,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[]
2,e-0-4c633939e1621b8d48842553-2,0,"[{'summary': 'This spot is popular', 'type': '...",4c633939e1621b8d48842553,Subway,6210 Finch Ave. W Suite 103,at Albion Rd.,43.742421,-79.589471,"[{'label': 'display', 'lat': 43.74242142838322...",344,M9V 3Y5,CA,Etobicoke,ON,Canada,"[6210 Finch Ave. W Suite 103 (at Albion Rd.), ...","[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",0,[]
3,e-0-4be70e26cf200f47e334153c-3,0,"[{'summary': 'This spot is popular', 'type': '...",4be70e26cf200f47e334153c,Popeyes Louisiana Kitchen,1530 Albion Rd.,at Kipling Ave. (Albion Centre),43.741202,-79.584545,"[{'label': 'display', 'lat': 43.74120165509377...",370,M9V 1B4,CA,Etobicoke,ON,Canada,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",0,[]
4,e-0-4cd4738cdfb4a1cd4337535c-4,0,"[{'summary': 'This spot is popular', 'type': '...",4cd4738cdfb4a1cd4337535c,The Beer Store,1530 Albion Rd,Near Finch Ave. W.,43.741694,-79.584373,"[{'label': 'display', 'lat': 43.7416936, 'lng'...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",0,[]
5,e-0-4d8ba6910c4e41bdaaf7667f-5,0,"[{'summary': 'This spot is popular', 'type': '...",4d8ba6910c4e41bdaaf7667f,Pizza Pizza,"1530 Albion Road, Unit T25",NaN,43.741569,-79.584489,"[{'label': 'display', 'lat': 43.74156896801906...",397,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[]
6,e-0-5112b872e4b0c0a78d7bcd27-6,0,"[{'summary': 'This spot is popular', 'type': '...",5112b872e4b0c0a78d7bcd27,Sunny Foodmart,1620 Albion road,Albion Road and Finch Ave,43.741840,-79.590561,"[{'label': 'display', 'lat': 43.74184023292111...",319,NaN,CA,Toronto,ON,Canada,"[1620 Albion road (Albion Road and Finch Ave),...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
7,e-0-4cd9d00734bb8cfa6576babf-7,0,"[{'summary': 'This spot is popular', 'type': '...",4cd9d00734bb8cfa6576babf,Tim Hortons,6220 Finch Ave West,Albion,43.742015,-79.589690,"[{'label': 'display', 'lat': 43.7420146612893,...",306,M9V 0A1,CA,Etobicoke,ON,Canada,"[6220 Finch Ave West (Albion), Etobicoke ON M9...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[]
8,e-0-4c1951d6834e2d7f2d3a2a80-8,0,"[{'summary': 'This spot is popular', 'type': '...",4c1951d6834e2d7f2d3a2a80,McDonald's,1530 Albion Road,NaN,43.741635,-79.584446,"[{'label': 'display', 'lat': 43.74163461048035...",404,M9V 1B4,CA,Toronto,ON,Canada,"[1530 Albion Road, Toronto ON M9V 1B4, Canada]","[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[]
9,e-0-4f035c5c5c5c51dd31b52e55-9,0,"[{'summary': 'This spot is popular', 'type': '...",4f035c5c5c5c51dd31b52e55,Dollarama,"1620 Albion Rd,Finch Albion Centre",NaN,43.742018,-79.591076,"[{'label': 'display', 'la

In [34]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sheriff's No Frills,Grocery Store,43.741968,-79.586639
1,Shoppers Drug Mart,Pharmacy,43.740832,-79.583347
2,Subway,Sandwich Place,43.742421,-79.589471
3,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
4,The Beer Store,Beer Store,43.741694,-79.584373


In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

11 venues were returned by Foursquare.


In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


In [ ]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York


In [ ]:
print(venues.shape)
venues.head()

In [ ]:
venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

In [ ]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

In [ ]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!